In [2]:
import pandas as pd
import numpy as np

In [ ]:
first_data = pd.read_csv('data/1995_Street_Tree_Census.csv')
second_data = pd.read_csv('data/2005_Street_Tree_Census.csv')
third_data = pd.read_csv('data/2015_Street_Tree_Census.csv')

# Standardize feature columns across datasets

In [21]:
# Extract the columns we care about
first_data = first_data[["Latitude", "Longitude", "Borough", "Postcode_Original", "Spc_Latin", "Diameter", "Condition", "Wires", "Sidewalk_Condition"]]
second_data = second_data[["latitude", "longitude", "boroname", "zipcode", "spc_latin", "tree_dbh", "status", "wire_htap", "wire_prime", "wire_2nd", "wire_other", "inf_wires", "sidw_crack", "sidw_raise"]]
third_data = third_data[["latitude", "longitude", "borough", "postcode", "spc_latin", "tree_dbh", "status", "health"]]

In [22]:
# Remove spurious NAs
first_data = first_data[first_data['Postcode_Original'].notna()]


In [23]:
# Consolidate the sidewalk and wire data
second_data['wires'] = np.where(((second_data['wire_htap'] == 'Yes') | (second_data['wire_prime'] == 'Yes') | (second_data['wire_2nd'] == 'Yes') | (second_data['wire_other'] == 'Yes') | (second_data['inf_wires'] == 'Yes')), 1, 0)
second_data['sidew_crack_raise'] = np.where(((second_data['sidw_crack'] == 'Yes') | (second_data['sidw_raise'] == 'Yes')), 1, 0)
second_data.drop('wire_htap', inplace=True, axis=1)
second_data.drop('wire_prime', inplace=True, axis=1)
second_data.drop('wire_2nd', inplace=True, axis=1)
second_data.drop('wire_other', inplace=True, axis=1)
second_data.drop('inf_wires', inplace=True, axis=1)
second_data.drop('sidw_crack', inplace=True, axis=1)
second_data.drop('sidw_raise', inplace=True, axis=1)

first_data['Wires'] = np.where(((first_data['Wires'] == "Yes") | (first_data['Wires'] == "Secondary")), 1, 0)
first_data['Sidewalk_Condition'] = np.where(((first_data['Sidewalk_Condition'] == "Raised")), 1, 0)



In [ ]:
# Rename columns
first_data = first_data.rename(columns={"Postcode_Original": "zipcode", "Borough": "borough", "Spc_Latin": "spc_latin", "Diameter": "tree_diameter", "Wires": "wires", "Sidewalk_Condition": "sidew_crack_raise", 'Latitude': 'latitude', 'Longitude': 'longitude'})
second_data = second_data.rename(columns={"boroname": "borough", "tree_dbh": "tree_diameter"})
second_data.borough[second_data['borough'] == 5] = 'Staten Island'
third_data = third_data.rename(columns={'postcode': 'zipcode'})


In [25]:
# Lowercase all species names
first_data = first_data[first_data['spc_latin'].notna()]
first_data['spc_latin'] = first_data['spc_latin'].str.lower()
second_data = second_data[second_data['spc_latin'].notna()]
second_data['spc_latin'] = second_data['spc_latin'].str.lower()


# Standardize all health statuses across the 2005 and 2015 datasets

In [ ]:
# Rename "Excellent" to "Good", rename "Good" to "Fair" (bc 2005 data had different naming scheme for the same 4 health levels)
second_data.status[second_data['status'] == "Good"] = "Fair"
second_data.status[second_data['status'] == "Excellent"] = "Good"
second_data = second_data.rename(columns={"status": "health"})
second_data.health.unique()



In [ ]:
# Set all status to be "Alive" or "Dead" (remove "Stump")
third_data.health[(third_data['health'].isna()) & (third_data['status'] != "Alive")] = "Dead"
# Remove the row with status == alive and health is na
third_data.drop(third_data[third_data['health'].isna()].index, inplace = True)
# Drop extra status column
third_data.drop(columns=['status'], inplace=True)


In [28]:
# Make categorical health data numeric
health_dict = {'Dead':0, 'Poor':1, 'Fair':2, 'Good':3}
second_data['health_status'] = second_data['health'].map(health_dict)
third_data['health_status'] = third_data["health"].map(health_dict)
second_data

,latitude,longitude,borough,zipcode,spc_latin,tree_diameter,health,wires,sidew_crack_raise,health_status
0,40.632653,-74.000245,Brooklyn,11219,pyrus calleryana,6,Fair,1,0,2
1,40.620084,-73.901453,Brooklyn,11234,platanus acerifolia,6,Fair,0,0,2
2,40.617996,-73.899111,Brooklyn,11234,acer platanoides crimson king,13,Fair,1,0,2
3,40.619694,-73.901003,Brooklyn,11234,platanus acerifolia,13,Fair,0,0,2
4,40.618323,-73.899467,Brooklyn,11234,platanus acerifolia,15,Fair,0,0,2
...,...,...,...,...,...,...,...,...,...,...
592367,40.586260,-74.148797,Staten Island,10314,platanus acerifolia,9,Fair,0,0,2
592368,40.586090,-74.149013,Staten Island,10314,acer saccharinum,7,Fair,0,0,2
592369,40.585802,-74.149156,Staten Island,10314,acer platanoides crimson king,5,Fair,0,0,2
592370,40.585802,-74.149156,Staten Island,10314,acer platanoides crimson king,9,Fair,0,0,2


# Group trees into geospatial segments that match across datasets

In [29]:
# Group trees into geospatial segments by lat and long
step = 0.0005 # This is about 1/4 block ish, 50 meters at equator
to_bin = lambda x: np.floor(x / step) * step
second_data["latBin"] = to_bin(second_data.latitude)
second_data["lonBin"] = to_bin(second_data.longitude)

# Drop all rows where lat or long is 0
second_data.drop(second_data[second_data['latBin'] == 0].index, inplace = True)
second_data.drop(second_data[second_data['lonBin'] == 0].index, inplace = True)


In [30]:
# Only keep the rows whose zipcodes overlap across datasets
zipcodes_intersection = set(first_data['zipcode']).intersection(set(second_data['zipcode'])).intersection(third_data['zipcode'])
first_data = first_data[first_data['zipcode'].isin(zipcodes_intersection)]
second_data = second_data[second_data['zipcode'].isin(zipcodes_intersection)]
third_data = third_data[third_data['zipcode'].isin(zipcodes_intersection)]


In [31]:
# Set latBin and lonBin values to match across second and third datasets
from scipy.spatial.distance import cdist

third_data["latBin"] = 0
third_data["lonBin"] = 0
second_data = second_data.sort_values('zipcode')
third_data = third_data.sort_values('zipcode')

latBins = []
lonBins = []
for zipcode in sorted(list(zipcodes_intersection)):
    temp_second = second_data[second_data['zipcode'] == zipcode]
    temp_third = third_data[third_data['zipcode'] == zipcode]
    temp = temp_second[['latBin','lonBin']].iloc[np.argmin(cdist(temp_third[['latitude','longitude']], temp_second[['latBin','lonBin']], metric='euclidean'), axis=1)]
    latBins.extend(list(temp.latBin))
    lonBins.extend(list(temp.lonBin))
third_data['latBin'] = latBins
third_data['lonBin'] = lonBins


In [32]:
# Do the same for the first data
first_data["latBin"] = 0
first_data["lonBin"] = 0
second_data = second_data.sort_values('zipcode')
first_data = first_data.sort_values('zipcode')

latBins = []
lonBins = []
for zipcode in sorted(list(zipcodes_intersection)):
    temp_second = second_data[second_data['zipcode'] == zipcode]
    temp_one = first_data[first_data['zipcode'] == zipcode]
    temp = temp_second[['latBin','lonBin']].iloc[np.argmin(cdist(temp_one[['latitude','longitude']], temp_second[['latBin','lonBin']], metric='euclidean'), axis=1)]
    latBins.extend(list(temp.latBin))
    lonBins.extend(list(temp.lonBin))
first_data['latBin'] = latBins
first_data['lonBin'] = lonBins


In [33]:
# Drop the rows whose actual lat and long differ too much from their latBin and lonBin

third_data['lonDistance'] = (third_data['lonBin']-third_data['longitude']).abs()
third_data['latDistance'] = (third_data['latBin']-third_data['latitude']).abs()

third_data.drop(third_data[third_data['health'].isna()].index, inplace = True)

third_data.drop(third_data[third_data['lonDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia, or 111 meters @ equator
third_data.drop(third_data[third_data['latDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia, or 111 meters @ equator


first_data['lonDistance'] = (first_data['lonBin']-first_data['longitude']).abs()
first_data['latDistance'] = (first_data['latBin']-first_data['latitude']).abs()

first_data.drop(first_data[first_data['lonDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia, or 111 meters @ equator
first_data.drop(first_data[first_data['latDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia, or 111 meters @ equator


# Merge the three datasets 
### Join the 1995 (first_data) features to the 2005 (second_data) health status values (matching on latBin and lonBin)
### Join the 2005 features to the 2015 health status values (matching on latBin and lonBin)

In [34]:
# Average the health data based on latBin/lonBin groups (and round to nearest whole number aka nearest health category)
second_data['avg_health_round'] = second_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean').round(0)
third_data['avg_health_round'] = third_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean').round(0)
second_data['avg_health'] = second_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean')
third_data['avg_health'] = third_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean')

In [35]:
# Keep only necessary features and merge dataframes

second_data_health = second_data.drop(columns={'tree_diameter', 'borough', 'health', 'health_status', 'latitude', 'longitude', 'spc_latin', 'zipcode', 'wires', 'sidew_crack_raise'})
first_data_features = first_data.drop(columns={'Condition'})
second_data_health = second_data_health.drop_duplicates(subset=['avg_health', 'avg_health_round', 'latBin','lonBin'])

merged_first_second = first_data_features.merge(second_data_health, on=['latBin', 'lonBin'], how="inner")
merged_first_second

third_data_health = third_data.drop(['borough', 'zipcode', 'spc_latin', 'tree_dbh', 'health', 'health_status', 'latitude', 'longitude'], axis=1)
second_data_features = second_data.drop(columns=['avg_health', 'avg_health_round', 'health', 'health_status'])
third_data_health = third_data_health.drop_duplicates(subset=['avg_health','avg_health_round', 'latBin','lonBin'])

merged_second_third = second_data_features.merge(third_data_health, on=['latBin', 'lonBin'], how="inner")
merged_second_third

,latitude,longitude,borough,zipcode,spc_latin,tree_diameter,wires,sidew_crack_raise,latBin,lonBin,lonDistance,latDistance,avg_health_round,avg_health
0,40.749276,-73.999314,Manhattan,10001,tilia cordata,6,0,0,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
1,40.749276,-73.999314,Manhattan,10001,robinia pseudoacacia,8,0,0,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
2,40.749276,-73.999314,Manhattan,10001,quercus palustris,9,0,0,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
3,40.749119,-73.999401,Manhattan,10001,platanus acerifolia,14,0,0,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
4,40.749320,-73.999282,Manhattan,10001,zelkova serrata,5,0,0,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544364,40.581102,-73.852954,Queens,11694,platanus acerifolia,16,0,1,40.581,-73.8530,1.771800e-04,0.000005,3.0,3.000000
544365,40.579157,-73.845045,Queens,11694,acer platanoides,11,1,1,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545
544366,40.579157,-73.845045,Queens,11694,acer platanoides,14,1,1,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545
544367,40.579157,-73.845045,Queens,11694,acer platanoides,14,1,0,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545


In [44]:
# Save dataset for use in models
final_data = merged_first_second.append(merged_second_third)
final_data.to_csv('data/parsed_data.csv')


# Parse PLUTO data and join with tree data

In [ ]:
pluto_data = pd.read_csv('data/Primary_Land_Use_Tax_Lot_Output__PLUTO_.csv')


In [144]:
pluto_data = pluto_data[['latitude', 'longitude', 'landuse', 'numfloors', 'yearbuilt', 'postcode']]
pluto_data = pluto_data.dropna()
pluto_data = pluto_data[pluto_data['yearbuilt'] <= 2005] # Anything after 2005 is built after the tree features we are using
pluto_data = pluto_data[pluto_data['yearbuilt'] > 0]
pluto_data

,latitude,longitude,landuse,numfloors,yearbuilt,postcode
0,40.733326,-73.991972,2.0,7.0,1900.0,10003.0
2,40.593135,-73.969560,1.0,2.0,1930.0,11223.0
3,40.715699,-73.803864,1.0,2.0,1935.0,11432.0
4,40.725978,-73.723547,1.0,2.0,1935.0,11426.0
5,40.679689,-73.856363,1.0,2.0,1920.0,11417.0
...,...,...,...,...,...,...
858981,40.593647,-74.083436,1.0,2.0,1970.0,10305.0
858995,40.564644,-74.099340,1.0,1.0,1920.0,10306.0
859000,40.569010,-74.143626,1.0,1.0,1960.0,10306.0
859001,40.571655,-74.128157,1.0,2.0,1965.0,10306.0


In [ ]:
# Set lat and lon bins for PLUTO data
# Using same process as with the tree datasets
# This throws out some PLUTO data, but seems to leave all tree data
final_tree_data = final_data
zipcodes_intersection = set(final_tree_data['zipcode']).intersection(set(pluto_data['postcode']))
tree_data = final_tree_data[final_tree_data['zipcode'].isin(zipcodes_intersection)]
pluto_data = pluto_data[pluto_data['postcode'].isin(zipcodes_intersection)]
pluto_data


pluto_data["latBin"] = 0
pluto_data["lonBin"] = 0
tree_data = tree_data.sort_values('zipcode')
pluto_data = pluto_data.sort_values('postcode')

latBins = []
lonBins = []
for zipcode in sorted(list(zipcodes_intersection)):
    temp_tree = tree_data[tree_data['zipcode'] == zipcode]
    temp_pluto = pluto_data[pluto_data['postcode'] == zipcode]
    temp = temp_tree[['latBin','lonBin']].iloc[np.argmin(cdist(temp_pluto[['latitude','longitude']], temp_tree[['latBin','lonBin']], metric='euclidean'), axis=1)]
    latBins.extend(list(temp.latBin))
    lonBins.extend(list(temp.lonBin))
pluto_data['latBin'] = latBins
pluto_data['lonBin'] = lonBins

In [146]:
# Again, drop any PLUTO data whose latbin and lonbins are too far from their real lat and lon
pluto_data['lonDistance'] = (pluto_data['lonBin']-pluto_data['longitude']).abs()
pluto_data['latDistance'] = (pluto_data['latBin']-pluto_data['latitude']).abs()

pluto_data.drop(pluto_data[pluto_data['lonDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia
pluto_data.drop(pluto_data[pluto_data['latDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia


In [147]:
# Average PLUTO data based on latBin/lonBin groups
pluto_data['landuseavg'] = pluto_data.groupby(["latBin", "lonBin"])['landuse'].transform('mean').round(0)
pluto_data['numfloorsavg'] = pluto_data.groupby(["latBin", "lonBin"])['numfloors'].transform('mean').round(0)
pluto_data

,latitude,longitude,landuse,numfloors,yearbuilt,postcode,latBin,lonBin,lonDistance,latDistance,landuseavg,numfloorsavg
752683,40.746341,-73.989945,4.0,4.0,1910.0,10001.0,40.7465,-73.9900,0.000055,0.000159,4.0,4.0
451155,40.745894,-73.991017,5.0,4.0,1930.0,10001.0,40.7455,-73.9910,0.000017,0.000394,4.0,7.0
543775,40.745836,-73.991367,4.0,4.0,1910.0,10001.0,40.7455,-73.9915,0.000133,0.000336,4.0,10.0
198710,40.746679,-73.992620,3.0,7.0,1910.0,10001.0,40.7470,-73.9920,0.000620,0.000321,4.0,7.0
227122,40.747008,-73.996081,10.0,6.0,1930.0,10001.0,40.7465,-73.9960,0.000081,0.000509,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
824399,40.578259,-73.851012,1.0,2.0,1940.0,11694.0,40.5780,-73.8515,0.000488,0.000259,2.0,2.0
631655,40.581107,-73.830366,4.0,2.0,1930.0,11694.0,40.5810,-73.8305,0.000134,0.000107,1.0,1.0
824405,40.573519,-73.853276,1.0,2.5,1930.0,11694.0,40.5735,-73.8530,0.000276,0.000019,1.0,2.0
824417,40.570528,-73.858077,1.0,2.0,1930.0,11694.0,40.5705,-73.8580,0.000077,0.000028,1.0,2.0


In [ ]:
# Prepare to merge PLUTO data with tree data
pluto_data = pluto_data.drop(columns=['numfloors', 'landuse', 'longitude', 'latitude', 'latDistance', 'lonDistance', 'postcode', 'yearbuilt'])
pluto_data = pluto_data.drop_duplicates(subset=['landuseavg','numfloorsavg'])


In [ ]:
# Merge PLUTO data with tree data
merge_tree_pluto = tree_data.merge(pluto_data, on=['latBin', 'lonBin'], how="inner")
merge_tree_pluto

In [176]:
# Save merged PLUTO and tree data
merge_tree_pluto.to_csv('data/parsed_pluto_tree_data.csv')
